In [5]:
from __future__ import division
import os, sys
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches

import pandas as pd
from svgpathtools import parse_path, wsvg, svg2paths

### setup

In [6]:
# directory & file hierarchy
proj_dir = os.path.abspath('../../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis','python') not in sys.path:
    sys.path.insert(1,os.path.join(proj_dir,'analysis','python'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

import svg_rendering_helpers as srh
reload(srh)

<module 'svg_rendering_helpers' from '/Users/judithfan/graphical_conventions/analysis/python/svg_rendering_helpers.pyc'>

In [7]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['3dObjects']
coll = db['graphical_conventions']

# which iteration name should we use?
## "good" iterations that we currently combine: ['run3_size4_waiting', 'run4_generalization']
iterationNames = ['run3_size4_waiting', 'run4_generalization']

### svg rendering

In [8]:
# list of all gameids across iterationNames
gameids = coll.find({'iterationName': {'$in': iterationNames}}).distinct('gameid')

In [9]:
# sample game
this_game = gameids[10]

In [10]:
# sample stroke recs from a trial
this_trial = 2
stroke_recs = coll.find({ '$and': [{'gameid': this_game}, 
                                   {'eventType': 'stroke'}, 
                                   {'trialNum':this_trial}]}).sort('time')

In [11]:
# get list of svgs
svg_list = srh.make_svg_list(stroke_recs)

In [12]:
# grab sample stroke
this_stroke = svg_list[0]

In [13]:
parsed1 = parse_path(svg_list[0])
parsed2 = parse_path(svg_list[1])

In [14]:
## list of paths to render out in same svg
parsed = [parsed1,parsed2]

#### wsvg arguments
```
wsvg(paths=None, colors=None, filename=join, stroke_widths=None, nodes=None, node_colors=None, node_radii=None, openinbrowser=False, timestamp=False, margin_size=0.1, mindim=600, dimensions=None, viewbox=None, text=None, text_path=None, font_size=None, attributes=None, svg_attributes=None)
```

### what we need (tests on single sketches)

- render_svg
- svg_to_png
- svg2paths (already defined in svgpathtools)

In [15]:
reload(srh)
## render out svg
srh.render_svg(parsed,base_dir=sketch_dir)

Rendering out to svg/tmp.svg


In [16]:
## get svg path list
svg_paths = srh.generate_svg_path_list(os.path.join(sketch_dir,'svg'))

In [17]:
## convert to png
srh.svg_to_png(svg_paths,base_dir=sketch_dir)

convert /Users/judithfan/graphical_conventions/sketches/svg/tmp.svg /Users/judithfan/graphical_conventions/sketches/png/Users.png


8

### scratch space for scaling up to render out whole dataset 

In [20]:
svg_list

[u'M60,178c0,29.70911 -9.59322,58.1695 -16,87c-0.65296,2.93834 -2.01484,21 -5,21',
 u'M61,177c39.5763,0 79.60696,5.71725 119,9c5.5946,0.46622 35,4.3235 35,1',
 u'M215,189c0,31.52135 0.18595,61.88431 -5,93c-2.32692,13.96149 -3,27.87705 -3,42c0,6.81803 -3,1.77014 -3,-2',
 u'M53,151c0,13.50457 28.31649,25.15824 38,30c19.67362,9.83681 111.5677,14.64846 126,-7c4.73132,-7.09698 4,-21.42011 4,-30',
 u'M98,57c0,31.97249 4.04764,61.38115 8,93c0.77779,6.2223 1.06638,30 4,30',
 u'M163,59c0,27.06224 -2.24522,55.05739 0,82c1.12236,13.46829 8,27.83413 8,41',
 u'M201,73c0,34.92156 2,68.27949 2,103',
 u'M67,57c13.21193,0 25.01952,-2.11545 38,-5c23.61457,-5.24768 49.7106,-8.42894 74,-6c9.00189,0.90019 56,21.88598 56,31']

In [21]:
svg_list_2 = [u'M87,105c0,-10.99517 24.80021,-15 32,-15c18.28964,0 41.9874,-5.00252 62,-1c6.36137,1.27227 13.72953,13 18,13',
 u'M109,94c0,-3.93189 7,6.18808 7,10c0,10.12106 4,16.33181 4,27',
 u'M165,92c0,-2.38519 4.92395,11.78271 5,12c3.95196,11.29132 7,27.32601 7,38']

In [22]:
parsed3 = [parse_path(i) for i in svg_list_2]


In [23]:
parsed3

[Path(CubicBezier(start=(87+105j), control1=(87+94.00483j), control2=(111.80021+90j), end=(119+90j)),
      CubicBezier(start=(119+90j), control1=(137.28964+90j), control2=(160.9874+84.99748j), end=(181+89j)),
      CubicBezier(start=(181+89j), control1=(187.36137+90.27227j), control2=(194.72953+102j), end=(199+102j))),
 Path(CubicBezier(start=(109+94j), control1=(109+90.06811j), control2=(116+100.18808j), end=(116+104j)),
      CubicBezier(start=(116+104j), control1=(116+114.12106j), control2=(120+120.33181j), end=(120+131j))),
 Path(CubicBezier(start=(165+92j), control1=(165+89.61481j), control2=(169.92395+103.78271j), end=(170+104j)),
      CubicBezier(start=(170+104j), control1=(173.95196+115.29132j), control2=(177+131.32601j), end=(177+142j)))]

In [24]:
srh.render_svg(parsed3,base_dir=sketch_dir)

Rendering out to svg/tmp.svg
